> **Student Edition (W02B)**  
> - Ejecuta los **DEMO** como guía.  
> - En **TU TURNO (1–4)** encontrarás `TODO`: debes escribir la consulta.  
> - Regla de oro: antes de un JOIN, verifica **grain** y **cardinalidad** (si no, duplicas filas).

# W03 – SQL esencial II (JOINs + CTEs) y cardinalidad práctica

## Conexión con DDIA
- **DDIA Cap. 2**: modelado + consultas; por qué las relaciones importan.
- Conexión práctica con **Cap. 3**: algunos joins se vuelven caros o peligrosos si la cardinalidad no está controlada.

## Prerrequisitos
- W02A (o dominar SELECT/WHERE/GROUP BY).
- Tener `raw_ps` disponible (el notebook puede descargar si falta).

## Objetivos
- Construir dimensiones simples (`dim_host`, `dim_discovery`).
- Usar `JOIN` (INNER/LEFT) y **demostrar** problemas de cardinalidad.
- Usar `CTE` (`WITH ...`) para estructurar consultas.
- Validar joins con conteos: evitar duplicación accidental.

## Checklist de evidencias
- [ ] Creaste `dim_host` y `dim_discovery`
- [ ] Mostraste un JOIN “malo” (duplica filas) + corrección
- [ ] 4 consultas del TU TURNO completas


In [50]:
# Setup común (cross-platform)
import sys, subprocess
from pathlib import Path
import duckdb

DB_PATH = Path("data/exoplanets.duckdb")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
con = duckdb.connect(str(DB_PATH))

def run_module(mod: str, *args: str):
    cmd = [sys.executable, "-m", mod, *args]
    print("Running:", " ".join(cmd))
    subprocess.check_call(cmd)

raw_csv = Path("data/raw/pscomppars.csv")
if not raw_csv.exists():
    run_module("src.ingest.download_exoplanets", "--format", "csv", "--limit", "50000")

# DuckDB no permite parámetros preparados en DDL (ej. CREATE VIEW).
# Insertamos la ruta como literal SQL, escapando comillas simples.
def sql_quote(s: str) -> str:
    return "'" + s.replace("'", "''") + "'"

raw_csv_abs = raw_csv.resolve()
con.execute(
    f"CREATE OR REPLACE VIEW raw_ps AS SELECT * FROM read_csv_auto({sql_quote(raw_csv_abs.as_posix())})"
)
con.sql("SELECT count(*) AS n_rows FROM raw_ps").show()


┌────────┐
│ n_rows │
│ int64  │
├────────┤
│   6065 │
└────────┘



In [49]:
# (Opcional) Ver columnas disponibles en raw_ps (útil para depurar)
con.sql("DESCRIBE raw_ps").show()

┌─────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name   │ column_type │  null   │   key   │ default │  extra  │
│     varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ pl_name         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ hostname        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ discoverymethod │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ disc_year       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ sy_snum         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ sy_pnum         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ sy_dist         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ ra              │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ dec             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ pl_orbper       │ DOUBLE      │ YES 

## DEMO (docente)


In [52]:
# DEMO 1: dimensión de hosts (1 fila por hostname) SOLO con SQL básico
# Clave: hostname
#
# Importante:
# - DISTINCT sobre (hostname, ra, ...) NO garantiza 1 fila por hostname.
# - Para forzar 1 fila por hostname SIN window functions, agregamos por hostname.
# - MAX(...) es una forma simple de "escoger un valor" y además ignora NULLs.
#
# Nota pedagógica: ejemplo didáctico. Más adelante veremos políticas de resolución más robustas.

con.execute("""
CREATE OR REPLACE TABLE dim_host_ra AS
SELECT
  hostname,
  MAX(ra) AS ra
FROM raw_ps
WHERE hostname IS NOT NULL
GROUP BY hostname
""")

# Validación rápida: en una dimensión correcta, n_rows == n_keys
con.sql("SELECT COUNT(*) AS n_rows, COUNT(DISTINCT hostname) AS n_keys FROM dim_host_ra").show()

┌────────┬────────┐
│ n_rows │ n_keys │
│ int64  │ int64  │
├────────┼────────┤
│   4524 │   4524 │
└────────┴────────┘



In [53]:
con.execute("""
CREATE OR REPLACE TABLE dim_discovery AS
SELECT DISTINCT discoverymethod, disc_year
FROM raw_ps
""")
con.execute("SELECT count(*) FROM dim_discovery").fetchall()


[(137,)]

In [54]:
# DEMO 2: fact (grano = 1 fila por planeta)
con.execute("""
CREATE OR REPLACE TABLE fact_planet_raw AS
SELECT
  pl_name,
  hostname,
  discoverymethod,
  disc_year,
  pl_orbper,
  pl_rade,
  pl_bmasse,
  pl_eqt
FROM raw_ps
WHERE pl_name IS NOT NULL
""")
con.sql("SELECT count(*) FROM fact_planet_raw").show()


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         6065 │
└──────────────┘



### DEMO 3: JOIN correcto (many-to-one)
`fact_planet_raw` → `dim_host` debería ser muchos-a-uno. Si `dim_host` tiene hostname único, **no debe multiplicar filas**.


In [57]:
n_fact = con.execute("SELECT count(*) FROM fact_planet_raw").fetchone()[0]
n_join = con.execute("""
SELECT count(*)
FROM fact_planet_raw f
JOIN dim_host_ra h
  ON f.hostname = h.hostname
""").fetchone()[0]
n_fact, n_join


(6065, 6065)

### DEMO 4: JOIN “malo” (duplica filas)
Error común: unirse a una tabla que **no es dimensión** (llave no única). Fabricamos una “dimensión mala” a propósito.


In [59]:
# DEMO 3: una "dimensión" MAL construida (violando 1 fila por hostname)
# Aquí NO deduplicamos: tendrá múltiples filas por hostname.
con.execute("""
CREATE OR REPLACE TABLE dim_host_bad AS
SELECT hostname, ra
FROM raw_ps
WHERE hostname IS NOT NULL
""")

# Evidencia sin HAVING (solo CTE + WHERE)
con.sql("""
WITH c AS (
  SELECT hostname, COUNT(*) AS cnt
  FROM dim_host_bad
  GROUP BY hostname
)
SELECT * FROM c
WHERE cnt > 1
ORDER BY cnt DESC
LIMIT 10
""").show()

┌────────────┬───────┐
│  hostname  │  cnt  │
│  varchar   │ int64 │
├────────────┼───────┤
│ KOI-351    │     8 │
│ TRAPPIST-1 │     7 │
│ HD 10180   │     6 │
│ K2-138     │     6 │
│ TOI-1136   │     6 │
│ Kepler-80  │     6 │
│ Kepler-11  │     6 │
│ HD 34445   │     6 │
│ HD 110067  │     6 │
│ HIP 41378  │     6 │
├────────────┴───────┤
│ 10 rows  2 columns │
└────────────────────┘



In [60]:
n_join_bad = con.execute("""
SELECT count(*)
FROM fact_planet_raw f
JOIN dim_host_bad h
  ON f.hostname = h.hostname
""").fetchone()[0]

n_fact, n_join_bad


(6065, 10705)

### DEMO 5: arreglar JOIN malo con CTE (deduplicación)


In [62]:
n_join_fixed = con.execute("""
WITH dim_host_fixed AS (
  SELECT DISTINCT hostname
  FROM dim_host_bad
)
SELECT count(*)
FROM fact_planet_raw f
JOIN dim_host_fixed h
  ON f.hostname = h.hostname
""").fetchone()[0]
n_fact, n_join_fixed


(6065, 6065)

## TU TURNO (práctica guiada)


### 1) LEFT JOIN y no-match: ¿cuántas filas quedan sin match en dim_host?

In [ ]:
# TODO (1) LEFT JOIN y no-match
# Objetivo: ¿cuántas filas de fact_planet quedan SIN match en dim_host?
# Pistas:
# - Usa LEFT JOIN fact_planet f con dim_host h ON f.hostname = h.hostname
# - Cuenta las filas donde h.hostname IS NULL
query = """
-- TODO: escribe tu SQL aquí
"""
con.sql(query).show()

┌───────┬──────────┐
│ total │ no_match │
│ int64 │  int128  │
├───────┼──────────┤
│  6065 │        0 │
└───────┴──────────┘



### 2) CTE + ranking: por año, método #1 (más planetas)

In [ ]:
# TODO (2) CTE + ranking
# Objetivo: por cada disc_year, obtener el discoverymethod #1 (más planetas) y su conteo
# Pistas:
# - Agrupa por disc_year, discoverymethod y cuenta
# - Usa una ventana: ROW_NUMBER() OVER(PARTITION BY disc_year ORDER BY n DESC)
# - Filtra rn = 1
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 3) Validación de cardinalidad: ¿hay duplicados en (discoverymethod, disc_year) en dim_discovery?

In [ ]:
# TODO (3) Validación de cardinalidad
# Objetivo: detectar si hay duplicados en la clave (discoverymethod, disc_year) dentro de dim_discovery
# Pistas:
# - GROUP BY discoverymethod, disc_year
# - HAVING COUNT(*) > 1
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 4) JOIN + agregación: promedio de RA del host por método

In [ ]:
# TODO (4) JOIN + agregación
# Objetivo: para cada discoverymethod, contar planetas y calcular el promedio de RA del host.
# Nota: dim_host_ra solo tiene (hostname, ra). Por eso usamos ra (no st_teff).
# Pistas:
# - JOIN fact_planet_raw (f) con dim_host_ra (h) por hostname
# - Filtra discoverymethod y ra no nulos
# - GROUP BY discoverymethod
query = """
-- TODO: escribe tu SQL aquí
"""
con.sql(query).show()

## DEMO (capstone, opcional) — CTE + Window Functions (estilo “heroes”)

> **Esta sección es opcional y NO entra en el entregable obligatorio.**  
> La idea es cerrar la teoría con un ejemplo “potente” como el que vimos con `heroes/team`:
> - 1er CTE: limpiamos/filtramos filas (como `heroes_clean`)
> - 2do CTE: calculamos métricas por grupo con **window functions** y rankeamos (como `ranked`)
> - SELECT final: nos quedamos con **la fila `rn=1`** por grupo (el “top 1” por partición)

**Pregunta análoga (en exoplanetas):**  
Para cada `hostname` (sistema), encontrar el **planeta con mayor radio** (`pl_rade`) y además reportar:
- `avg_radius_in_system` (promedio de radios en el sistema)
- `planets_in_system` (cuántos planetas tiene el sistema)
- un atributo del host (`ra`, si existe)

In [ ]:
# CTE + Window Functions (capstone)
query = r'''
WITH planets_clean AS (
  SELECT
    f.pl_name,
    f.hostname,
    f.pl_rade,
    h.ra
  FROM fact_planet_raw f
  LEFT JOIN dim_host_ra h
    ON h.hostname = f.hostname
  WHERE f.hostname IS NOT NULL
    AND f.pl_rade IS NOT NULL
    AND f.pl_rade BETWEEN 0 AND 30   -- filtro tipo "edad entre 0 y 120"
),
ranked AS (
  SELECT
    hostname,
    ra,
    pl_name,
    pl_rade,
    AVG(pl_rade) OVER (PARTITION BY hostname) AS avg_radius_in_system,
    COUNT(*)     OVER (PARTITION BY hostname) AS planets_in_system,
    ROW_NUMBER() OVER (
      PARTITION BY hostname
      ORDER BY pl_rade DESC, pl_name ASC
    ) AS rn
  FROM planets_clean
)
SELECT
  hostname,
  ra,
  pl_name AS largest_planet,
  pl_rade AS largest_radius,
  avg_radius_in_system,
  planets_in_system
FROM ranked
WHERE rn = 1
ORDER BY planets_in_system DESC, hostname
LIMIT 50;
'''
con.sql(query).show()

In [72]:
try:
    con.close()
    print("DuckDB connection closed.")
except NameError:
    print("No connection named 'con' in this notebook.")

DuckDB connection closed.


## Para entregar (W03) 

### En clase
1) `docs/w03_sql_practice.md` con el análisis de `dim_host_bad`:
   - Los 4 conteos: `n_fact`, `n_join_good`, `n_join_bad`, `n_join_fixed` + 2–3 líneas explicando qué pasó.
   - Respuestas a **TODO 1–4** (cada una: SQL + output pegado).

2) `docs/decisions_log.md`: 1 entrada corta:
   - “Cómo validé cardinalidad antes de un JOIN” + evidencia (una query de duplicados o un conteo).

> **Nota:** La sección **DEMO capstone (CTE + Window)** es **solo demostración** (no se entrega).

### Tarea (para la próxima clase)
1) `docs/w03_join_case.md`: **1 caso real de JOIN malo**
   - evidencia con conteos antes/después
   - diagnóstico (qué clave falló)
   - fix simple (por ejemplo: dedupe con `GROUP BY`/`DISTINCT` o pre-agregación)

2) 2 consultas extra (en `docs/w03_sql_practice.md`):
   - 1 que incluya `JOIN`
   - 1 que incluya `CTE`